In [14]:
import graphlab as gl
import evaluate
import cPickle as pickle

In [15]:
ques = []
users = []
vals = []
with open('../train_data/localtraining3.txt', 'r') as f1:
    for line in f1:
        qid, uid, val = line.rstrip('\n').split()
        ques.append(qid)
        users.append(uid)
        vals.append(int(val))

        

In [16]:
quesv = []
usersv = []
#vals = []
with open('../train_data/localvalidation3.csv', 'r') as f1:
    line = f1.readline()
    for line in f1:
        qid, uid =  line.rstrip('\n').split(',')
        quesv.append(qid)
        usersv.append(uid)
        #vals.append(int(val))


In [17]:
sf = gl.SFrame({'user_id':users, 'item_id':ques, 'rating':vals})

In [18]:
sfv  = gl.SFrame({'user_id':usersv, 'item_id':quesv})

In [19]:
topics = []
with open('../train_data/user_info.txt', 'r') as f1:
    for line in f1:
        topic = map(str, (line.split()[1]).split('/'))
        topics.append(topic)

user_keys = pickle.load(open('../features/user_info_keys.dat', 'rb'))
ques_keys = pickle.load(open('../features/question_info_keys.dat', 'rb'))

#ufeat = []
#for i in range(len(topics)):
    #ufeat.append([1 if x in topics[i] else 0 for x in range(145)])
questopics = []
with open('../train_data/question_info.txt', 'r') as f1:
    for line in f1:
        questopics.append(line.split()[1])

    

In [20]:
user_info = gl.SFrame({'user_id': user_keys, 'categories':topics})
ques_info = gl.SFrame({'item_id': ques_keys, 'categories':questopics})
print ques_info

+------------+-------------------------------+
| categories |            item_id            |
+------------+-------------------------------+
|     0      | c1c0075239841777d5b01c40b3... |
|     1      | 367edcb36424493a7cf80f7090... |
|     2      | fb9e401d86f20205d97a25f3aa... |
|     3      | e153659c6c654cd12122232fca... |
|     1      | c1718491292fd44b9b33397d5e... |
|     4      | 85f22d70bd58c56708e9c3b276... |
|     5      | 3461879e8b67f1f9e08882f7f1... |
|     2      | 58695dc8b407850d8a028cec78... |
|     2      | 39b402c90bda0ba4b5fe125ac8... |
|     3      | 8304fb2d94318d4768acf96acc... |
+------------+-------------------------------+
[8095 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [216]:
m1 = gl.ranking_factorization_recommender.create(sf, target='rating', ranking_regularization=0.05, unobserved_rating_value=-0.6, solver='sgd')


Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 215033 observations with 26233 users and 7620 items.

Data prepared in: 0.343513s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0741844

Final training RMSE: 0.219806

In [217]:
res = m1.predict(sfv)

In [218]:
fname = 'tempfile'
print len(quesv)
with open(fname, 'w') as f1:
    f1.write('qid,uid,value\n')
    for i in range(len(quesv)):
        f1.write(quesv[i]+','+usersv[i]+','+str(res[i])+'\n')
print evaluate.ndcg(fname)
    

30719
0.470035359859
